# **Day 4**: *Ceres Search*

## Part 1

"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: `XMAS`.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of `XMAS` - you need to find all of them. Here are a few ways `XMAS` might appear, where irrelevant characters have been replaced with .:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

The actual word search will be full of letters instead. For example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

In this word search, `XMAS` occurs a total of `18` times; here's the same word search again, but where letters not involved in any `XMAS` have been replaced with `.`:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```

Take a look at the little Elf's word search. How many times does `XMAS` appear?

### Solution

In [ ]:
def find_m(matrix, row_limits, col_limits, row_idx, col_idx):
    """
    Find all directions relative to the current matrix indices where a value of
    "M" can be found. Returns a list of directions, where each direction looks
    like `direction = (delta_row_idx, delta_col_idx)`.
    """
    directions = []
    for i in range(row_idx-1, row_idx+2):
        if not row_limits[0] < i < row_limits[1]:
            continue

        for j in range(col_idx-1, col_idx+2):
            if not col_limits[0] < j < col_limits[1]:
                continue # out of bounds j

            if matrix[i][j] == "M":
                # get position of M relative to X
                directions.append((i - row_idx, j - col_idx))

    return directions


def count_xmas(matrix, row_limits, col_limits, row_idx, col_idx):
    """
    Count the number of XMAS sequences that can be found in the matrix given
    the current indices of the letter "X". First all directions with a value
    of "M" are found, then for each direction it is checked if values of "A"
    and "S" can be found in the same direction.

    Returns the number of XMAS sequences in any direction that can be found for
    a position (row_idx, col_idx) given that this position has a value of "X".
    """
    # find all directions in which the letter "M" is found
    directions = find_m(matrix, row_limits, col_limits, row_idx, col_idx)

    XMAS_count = 0

    # check each direction
    for direction in directions:
        delta_row, delta_col = direction

        # potential indices of XMAS
        X_idx = (row_idx, col_idx)
        M_idx = (X_idx[0] + delta_row, X_idx[1] + delta_col)
        A_idx = (M_idx[0] + delta_row, M_idx[1] + delta_col)
        S_idx = (A_idx[0] + delta_row, A_idx[1] + delta_col)

        # check if A and S indices are within bounds
        valid_bounds = (
            row_limits[0] < A_idx[0] < row_limits[1],
            col_limits[0] < A_idx[1] < col_limits[1],

            row_limits[0] < S_idx[0] < row_limits[1],
            col_limits[0] < S_idx[1] < col_limits[1],
        )

        # check if A and S indices contain the correct letters
        if all(valid_bounds):
            correct_letters = (
                matrix[A_idx[0]][A_idx[1]] == "A",
                matrix[S_idx[0]][S_idx[1]] == "S"
            )

            if all(correct_letters):
                XMAS_count += 1

    return XMAS_count


def my_func(input_string):
    lines = input_string.splitlines()
    matrix = [[val for val in line] for line in lines]

    row_limits = (-1, len(matrix))
    col_limits = (-1, len(matrix[0]))

    XMAS_count = 0
    for row_idx, row in enumerate(matrix):
        for col_idx, char in enumerate(row):
            if char == "X":
                XMAS_count += count_xmas(matrix, row_limits, col_limits, row_idx, col_idx)

    return XMAS_count

### Example

In [2]:
input_string = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""

my_func(input_string)


18

### Submission

In [3]:
with open('input/day04.txt', 'r') as file:
    input_string = file.read()

my_func(input_string)


2521

## Part 2

The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an `XMAS` puzzle; it's an `X`-`MAS` puzzle in which you're supposed to find two `MAS` in the shape of an `X`. One way to achieve that is like this:

```
M.S
.A.
M.S
```

Irrelevant characters have again been replaced with . in the above diagram. Within the `X`, each `MAS` can be written forwards or backwards.

Here's the same example from before, but this time all of the `X`-`MAS`es have been kept instead:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```

In this example, an `X`-`MAS` appears `9` times.

Flip the word search from the instructions back over to the word search side and try again. How many times does an `X`-`MAS` appear?

### Solution

In [4]:
def check_bounds(row_limits, col_limits, idx):
    """
    Given an index idx=(row_idx, col_idx), check if it is within the provided
    bounds.
    """
    return (
        row_limits[0] < idx[0] < row_limits[1]
        and col_limits[0] < idx[1] < col_limits[1]
    )


def check_corners(matrix, row_limits, col_limits, row_idx, col_idx):
    """
    Retrieve the values at the corners and check if each diagonal contains an
    "S" and an "M"
    """
    # corner indices
    TL = (row_idx-1, col_idx-1)
    TR = (row_idx-1, col_idx+1)
    BL = (row_idx+1, col_idx-1)
    BR = (row_idx+1, col_idx+1)

    # check if corners are within matrix bounds
    bounds_conditions = (
        check_bounds(row_limits, col_limits, TL),
        check_bounds(row_limits, col_limits, TR),
        check_bounds(row_limits, col_limits, BL),
        check_bounds(row_limits, col_limits, BR),
    )

    if not all(bounds_conditions):
        return False

    # get values at corners
    diag1_values = (matrix[TL[0]][TL[1]], matrix[BR[0]][BR[1]])
    diag2_values = (matrix[TR[0]][TR[1]], matrix[BL[0]][BL[1]])

    # check if both diagonals contain "S" and "M"
    value_conditions = (
        "M" in diag1_values and "M" in diag2_values,
        "S" in diag1_values and "S" in diag2_values,
    )

    if not all(value_conditions):
        return False

    return True


def my_func(input_string):
    lines = input_string.splitlines()
    matrix = [[val for val in line] for line in lines]

    row_limits = (-1, len(matrix))
    col_limits = (-1, len(matrix[0]))

    XMAS_count = 0
    for row_idx, row in enumerate(matrix):
        for col_idx, char in enumerate(row):
            if char == "A":
                valid_X_MAS = check_corners(matrix, row_limits, col_limits, row_idx, col_idx)
                if valid_X_MAS:
                    XMAS_count += 1
    return XMAS_count


### Example

In [5]:
input_string = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""

my_func(input_string)


9

### Submission

In [6]:
with open('input/day04.txt', 'r') as file:
    input_string = file.read()


my_func(input_string)


1912